$$\rho \left( \frac{\partial u}{\partial t} + u \cdot \nabla u \right) = \nabla \cdot \sigma (u,p)+f$$
$$\nabla \cdot u = 0$$

$$ \sigma (u,p) = 2 \mu \epsilon(u)-p I$$

$$\epsilon(u) = \frac{1}{2} (\nabla u + (\nabla u)^T)$$




In [14]:
from mpi4py import MPI
from dolfinx import mesh
import ufl
import numpy as np
from dolfinx import fem

class BoundaryCondition():
    def __init__(self, type, marker, values, ds,facet_tag):
        self._type = type
        if type == "Dirichlet":
            u_D = fem.Function(V)
            u_D.interpolate(values)
            facets = facet_tag.find(marker)
            dofs = fem.locate_dofs_topological(V, fdim, facets)
            self._bc = fem.dirichletbc(u_D, dofs)
        elif type == "Neumann":
                self._bc = ufl.inner(values, v) * ds(marker)
        elif type == "Robin":
            self._bc = values[0] * ufl.inner(u-values[1], v)* ds(marker)
        else:
            raise TypeError("Unknown boundary condition: {0:s}".format(type))
    @property
    def bc(self):
        return self._bc

    @property
    def type(self):
        return self._type

In [12]:
import numpy as np
from dolfinx import fem
from dolfinx import mesh
import ufl
from mpi4py import MPI
from petsc4py import PETSc

from basix.ufl import element



In [9]:
domain = mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)
t = 0
T = 10
num_steps = 500

dt = T / num_steps

#Elements
v_cg2 = element("Lagrange", domain.topology.cell_name(), 2, shape=(domain.geometry.dim, ))
s_cg1 = element("Lagrange", domain.topology.cell_name(), 1)
V = functionspace(mesh, v_cg2)
Q = functionspace(mesh, s_cg1)

u = TrialFunction(V)
v = TestFunction(V)
p = TrialFunction(Q)
q = TestFunction(Q)

#Boundaries
boundaries=[
    (1, lambda x: np.isclose(x[0],0)),
    (2, lambda x: np.isclose(x[0],1)),
    (3, lambda x: np.isclose(x[1],0)),
    (4, lambda x: np.isclose(x[1],1))
]

boundary_conditions = [BoundaryCondition("Dirichlet", 1, u_ex,ds,facet_tag),
                       BoundaryCondition("Dirichlet", 2, u_ex,ds,facet_tag),
                       BoundaryCondition("Robin", 3, (r, s),ds,facet_tag),
                       BoundaryCondition("Neumann", 4, g,ds,facet_tag)]

In [10]:
domain.topology.cell_name()



'triangle'

In [15]:
ufl.sym?

Signature: ufl.sym(A)
Docstring: Take the symmetric part of A.
File:      /dolfinx-env/lib/python3.12/site-packages/ufl/operators.py
Type:      function